Implementing the "hack" described in the paper  "Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing" (https://arxiv.org/abs/2406.08464) to generate an instruction dataset with the format:

{"instruction": What type of cloud is typically associated with thunderstorms?,<br>
"output": The type of cloud typically associated with thunderstorms is cumulonimbus.}

LLama 8B- Instruct model is used


Evaluate: Quality of instructions, Difficulty of instructions, Instruction SImilarity

In [11]:
#importing libraries
from tqdm import tqdm
import json

In [4]:
#ollama run llama3
#Verifying if Ollama 3 is running
import psutil
 
def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running
 
ollama_running = check_if_running("ollama")
 
if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [18]:
#Function to interact with Ollama via REST API

import urllib.request
 
def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat",role="user"):
    data = {
        "model": model,
        "seed": 123,        
        "temperature": 1.,
        "top_p":1,
        "messages": [
            {"role": role, "content": prompt}
        ]
    }
 
    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")
 
    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]
 
    return response_data

In [27]:
result = query_model("What do Llamas eat?")
print(result)

Llamas are ruminant animals, which means they have a four-chambered stomach and primarily feed on plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: High-quality hay, such as timothy or alfalfa, is a staple in many llama diets. They enjoy the sweet taste and texture of hay.
3. Fruits and vegetables: Llamas might snack on fruits like apples, carrots, and sweet potatoes. They also enjoy leafy greens like kale, spinach, and collard greens.
4. Grains: Some llamas may receive grains like oats, barley, or corn as part of their diet, but these should be given in moderation to avoid digestive issues.
5. Minerals: Llamas require access to mineral supplements, such as salt, calcium, and phosphorus, which are essential for maintaining strong bones and overall health.

In the wild, llamas might eat a wide variety of plants, including:

* Leaves
* Fruits
* Bark
* Twigs
* F

In [25]:
#Extract the instructions using the prompt template proposed in the paper
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [26]:
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

None


In [23]:
instruction

''